# Transformation between internal and cartesian coordinates

In [1]:
import chemcoord as cc
import time

In [2]:
water = cc.Cartesian.read_xyz('water_dimer.xyz', start_index=1).give_zmat()
small = cc.Cartesian.read_xyz('MIL53_small.xyz', start_index=1).give_zmat()

# Naming convention

The table which defines the used references of each atom will be called
**construction table**.

The contruction table of the zmatrix of the water dimer can be seen here:

In [4]:
water.loc[:, ['b', 'a', 'd']]

,b,a,d
1,-9223372036854775808,-9223372036854775805,-9223372036854775807
2,1,-9223372036854775805,-9223372036854775807
3,1,2,-9223372036854775807
4,2,1,3
5,4,2,1
6,4,5,2


The absolute references are indicated by magic numbers ``-(sys.maxsize + 1)``.

* The atom which is to be set in the reference of three other atoms, is denoted $i$.
* The bond-defining atom is represented by $b$.
* The angle-defining atom is represented by $a$.
* The dihedral-defining atom is represented by $d$.

# Mathematical introduction

It is advantageous to treat a zmatrix simply as recursive spherical coordinates.

The $(n + 1)$-th atom uses three of the previous $n$ atoms as reference.
Those three atoms ($b, a, d$) are spanning a coordinate system, if we require righthandedness.
If we express the position of the atom $i$ in respect to this locally spanned coordinate system using
spherical coordinates, we arrive at the usual definition of a zmatrix.


PS: The question about right- or lefthandedness is equivalent to specifying a direction of rotation.
Chemcoord uses of course the [IUPAC definition](https://goldbook.iupac.org/html/T/T06406.html).

#### Ideal case

The ideal (and luckily most common) case is, that $\vec{ib}$, $\vec{ba}$, and $\vec{ad}$ are linearly independent.
In this case there exist a bijective mapping between spherical coordinates and cartesian coordinates and all angles, positions... are well defined.

![Ideal case](ideal_coordinate.png "Ideal case")

#### Linear angle

One pathologic case appears, if $\vec{ib}$ and $\vec{ba}$ are linear dependent.

This means, that the angle in the zmatrix is either $0^\circ$ or $180^\circ$.
In this case there are infinitely many dihedral angles for the same configuration in cartesian space.
Or to say it in a more analytical way:
The transformation from spherical coordinates to cartesian coordinates is surjective, but not injective.

For nearly all cases (e.g. expressing the potential-hyper-surface in terms of internal coordinates), the surjectivity property is sufficient.

A lot of other problematic cases can be automatically solved by assigning a default value to the dihedral angle by definition ($0^\circ$ in the case of chemcoord).

Usually the user does not need to think about this case, which is automatically handled by chemcoord.

![Linear angle](linear_angle.png "Linear angle")

#### Linear reference

The real pathologic case appears, if the three reference atoms used by 

![Linear dihedral](linear_dihedral.png "Linear dihedral")

# Symbolic evaluation

It is possible to use symbolic expressions from sympy.

In [8]:
import sympy
sympy.init_printing()
d = sympy.Symbol('d')

In [23]:
symb_water = water.copy()

In [24]:
symb_water.safe_loc[4, 'bond'] = d

In [25]:
symb_water

,atom,b,bond,a,angle,d,dihedral
1,O,$\vec{0}$,0,$\vec{e_z}$,0.000000,$\vec{e_x}$,0.000000
2,H,1,0.910922,$\vec{e_z}$,56.385853,$\vec{e_x}$,0.000000
3,H,1,0.910922,2,107.000024,$\vec{e_x}$,0.000000
4,O,2,$d$,1,132.466298,3,343.244987
5,H,4,0.910922,2,132.466298,1,180.000000
6,H,4,0.910922,5,107.000024,2,163.244987


In [27]:
symb_water.subs(d, 2)

,atom,b,bond,a,angle,d,dihedral
1,O,$\vec{0}$,0.000000,$\vec{e_z}$,0.000000,$\vec{e_x}$,0.000000
2,H,1,0.910922,$\vec{e_z}$,56.385853,$\vec{e_x}$,0.000000
3,H,1,0.910922,2,107.000024,$\vec{e_x}$,0.000000
4,O,2,2.000000,1,132.466298,3,343.244987
5,H,4,0.910922,2,132.466298,1,180.000000
6,H,4,0.910922,5,107.000024,2,163.244987


In [28]:
for i in range(2, 5):
    water.subs(d, i).give_cartesian().view()
    time.sleep(1)

# Binary operators

### Mathematical Operations:

The general rule is that mathematical operations using the binary operators
``(+ - * /)`` and the unary operators ``(+ - abs)``
are only applied to the ``['bond', 'angle', 'dihedral']`` columns.

**Addition/Subtraction/Multiplication/Division**:
The most common case is to add another Zmat instance.
In this case it is tested, if the used references are the same.
Afterwards the addition in the ``['bond', 'angle', 'dihedral']`` columns
is performed.
If you add a scalar to a Zmat it is added elementwise onto the
``['bond', 'angle', 'dihedral']`` columns.
If you add a 3-dimensional vector, list, tuple... the first element of this
vector is added elementwise to the ``'bond'`` column of the
Zmat instance and so on.
The third possibility is to add a matrix with
``shape=(len(Zmat), 3)`` which is again added elementwise.
The same rules are true for subtraction, division and multiplication.

In [32]:
distortion = water.copy()
distortion.unsafe_loc[:, ['bond', 'angle', 'dihedral']] = 0
distortion.safe_loc[4, 'bond'] = d

In [35]:
water + distortion

,atom,b,bond,a,angle,d,dihedral
1,O,$\vec{0}$,0,$\vec{e_z}$,0.000000,$\vec{e_x}$,0.000000
2,H,1,0.910922,$\vec{e_z}$,56.385853,$\vec{e_x}$,0.000000
3,H,1,0.910922,2,107.000024,$\vec{e_x}$,0.000000
4,O,2,$d + 2.3512055093207$,1,132.466298,3,343.244987
5,H,4,0.910922,2,132.466298,1,180.000000
6,H,4,0.910922,5,107.000024,2,163.244987


In [36]:
(water + distortion).subs(d, 3)

,atom,b,bond,a,angle,d,dihedral
1,O,$\vec{0}$,0.000000,$\vec{e_z}$,0.000000,$\vec{e_x}$,0.000000
2,H,1,0.910922,$\vec{e_z}$,56.385853,$\vec{e_x}$,0.000000
3,H,1,0.910922,2,107.000024,$\vec{e_x}$,0.000000
4,O,2,5.351206,1,132.466298,3,343.244987
5,H,4,0.910922,2,132.466298,1,180.000000
6,H,4,0.910922,5,107.000024,2,163.244987


In [37]:
(water + distortion).subs(d, 3).give_cartesian().view()